## import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from random import randint

## nn.Linear

layer

In [ ]:
def forward(x, l1, l2):
    return F.tanh(l2.forward(F.tanh(l1.forward(x))))

In [ ]:
layer1 = nn.Linear(3, 2)
layer2 = nn.Linear(2, 1)

In [ ]:
layer1.weight

Parameter containing:
tensor([[-0.1973,  0.5563,  0.0563],
        [-0.0590, -0.2371, -0.5569]], requires_grad=True)

In [ ]:
layer1.bias

Parameter containing:
tensor([ 0.1195, -0.0674], requires_grad=True)

In [ ]:
w1 = torch.tensor([[ 0.4476, -0.9389,  0.4359, -0.2581],
                            [-0.7970,  1.6001, -0.8061,  0.6937]])
w2 = torch.tensor([ 1.2324, -2.2670, -0.3373])

layer1.weight.data = w1[:, :3]
layer2.weight.data = w2[:2]

layer1.bias.data = w1[:, 3]
layer2.bias.data = w2[2]

In [ ]:
x = torch.FloatTensor([1, -1, 1])
y = forward(x, layer1, layer2)

In [ ]:
print(y.data)

tensor(0.9953)


## nn.Module

In [ ]:
class NetWork(nn.Module):
  def __init__(self, input_dim, num_hidden, output_dim):
    super().__init__()
    self.layer1 = nn.Linear(input_dim, num_hidden)
    self.layer2 = nn.Linear(num_hidden, output_dim)

  def forward(self, x):
    return F.tanh(self.layer2(F.tanh(self.layer1(x))))

In [ ]:
model = NetWork(3, 2, 1)

In [ ]:
model

NetWork(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2046,  0.4974, -0.3150],
         [-0.4507,  0.5590, -0.0701]], requires_grad=True),
 Parameter containing:
 tensor([-0.0372, -0.0314], requires_grad=True),
 Parameter containing:
 tensor([[-0.6978, -0.2416]], requires_grad=True),
 Parameter containing:
 tensor([0.3293], requires_grad=True)]

In [ ]:
X_train = torch.FloatTensor([(-1,-1,-1),
 (-1,-1,1), (-1,1,-1), (1,-1,-1),
 (1,1,-1), (1,-1,1), (-1,1,1), (1,1,1)])

y_train = torch.FloatTensor([-1, 1, -1, 1, -1, 1, -1, -1])
total = len(y_train)

In [ ]:
optimizer = optim.RMSprop(params=model.parameters(),lr=0.01)
loss_f = nn.MSELoss()

In [ ]:
model.train()

for _ in range(1000):
  k = randint(0, total-1)
  y = model(X_train[k])
  y = y.squeeze()

  loss = loss_f(y, y_train[k])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
model.eval()
for x, y in zip(X_train, y_train):
  y_pred = model(x)
  print(f'output {y_pred.data} -> {y}')

output tensor([-0.9840]) -> -1.0
output tensor([0.9891]) -> 1.0
output tensor([-0.9999]) -> -1.0
output tensor([0.9920]) -> 1.0
output tensor([-0.9998]) -> -1.0
output tensor([0.9998]) -> 1.0
output tensor([-0.9998]) -> -1.0
output tensor([-0.9918]) -> -1.0
